In [1]:
# import numpy as np
# import pandas as pd
# from scipy import stats
# import statsmodels.api as sm
# import statsmodels
# import matplotlib.pyplot as plt
# from UE_04_LinearRegDiagnostic import LinearRegDiagnostic

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import MNLogit

In [2]:
path = "C:/Users/david/E-15/Programmieren/MotorcycleClassification/"

In [3]:
df_train = pd.read_csv(path+"data/train/training_data.csv")
df_test = pd.read_csv(path+"data/validation/test_data.csv")

In [4]:
# Import data
y_cols = [
    "Category_ATV",
    "Category_Allround",
    "Category_Classic",
    "Category_Cross / motocross",
    "Category_Custom / cruiser",
    "Category_Enduro / offroad",
    "Category_Naked bike",
    "Category_Scooter",
    "Category_Sport",
    "Category_Sport touring",
    "Category_Super motard",
    "Category_Touring",
    "Category_Unspecified category"
]

# y: Labels
y_train_raw = df_train[y_cols]
y_test_raw = df_test[y_cols]
# X: Features (alles außer Kategorien)
X_train = df_train.drop(columns=y_cols)
X_test = df_test.drop(columns=y_cols)


In [5]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import MNLogit
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

# Annahme: Du hast bereits X_train, X_test, y_train, y_test

# 1. LabelEncoder erstellen und auf TRAIN-Daten fitten
le = LabelEncoder()

# y_train: One-Hot -> Single Column -> Encoded
y_train_single = y_train_raw.idxmax(axis=1)
y_train_single = y_train_single.str.replace('Category_', '')
y_train = le.fit_transform(y_train_single)

# y_test: One-Hot -> Single Column -> Encoded (mit dem GLEICHEN Encoder!)
y_test_single = y_test_raw.idxmax(axis=1)
y_test_single = y_test_single.str.replace('Category_', '')
y_test = le.transform(y_test_single)

print("Klassen-Mapping:")
for i, category in enumerate(le.classes_):
    print(f"{i}: {category}")

Klassen-Mapping:
0: ATV
1: Allround
2: Classic
3: Cross / motocross
4: Custom / cruiser
5: Enduro / offroad
6: Naked bike
7: Scooter
8: Sport
9: Sport touring
10: Super motard
11: Touring
12: Unspecified category


In [6]:
X_train.describe()

,"Engine type_In-line four, four-stroke","Engine type_In-line three, four-stroke","Engine type_Single cylinder, four-stroke","Engine type_Single cylinder, two-stroke","Engine type_Twin, four-stroke","Engine type_Twin, two-stroke","Engine type_Two cylinder boxer, four-stroke","Engine type_V2, four-stroke","Engine type_V4, four-stroke","Transmission type,final drive_Belt",...,Front brakes_Dual disc,Front brakes_Expanding brake,Front brakes_Expanding brake (drum brake),Front brakes_Single disc,Rear brakes_Expanding brake,Rear brakes_Expanding brake (drum brake),Rear brakes_Single disc,Displacement ccm,Fuel capacity liters,Power HP
count,13912.000000,13912.000000,13912.000000,13912.000000,13912.000000,13912.000000,13912.000000,13912.000000,13912.000000,13912.000000,...,13912.000000,13912.000000,13912.000000,13912.000000,13912.000000,13912.000000,13912.000000,13912.000000,13912.000000,13912.000000
mean,-0.000693,0.003792,-0.000930,-0.000014,0.003385,-0.003691,-0.001828,-0.001692,0.003659,0.002115,...,-0.000088,0.009807,-0.006304,-0.006118,-0.000919,-0.000785,0.001346,0.001748,0.003836,0.003485
std,0.999115,1.009566,0.999755,0.999987,1.004132,0.986980,0.995415,0.998834,1.014672,1.001570,...,0.999907,1.019293,0.991205,0.999787,0.999064,0.999497,0.999643,1.000286,1.002454,1.002374
min,-0.342927,-0.190116,-0.765012,-0.319963,-0.356792,-0.139893,-0.192215,-0.523949,-0.121952,-0.503665,...,-0.376761,-0.237079,-0.322901,-0.967510,-0.406922,-0.543061,-1.305322,-1.257002,-2.544231,-1.154505
25%,-0.342927,-0.190116,-0.765012,-0.319963,-0.356792,-0.139893,-0.192215,-0.523949,-0.121952,-0.503665,...,-0.376761,-0.237079,-0.322901,-0.967510,-0.406922,-0.543061,-1.305322,-0.948802,-0.723527,-0.827445
50%,-0.342927,-0.190116,-0.765012,-0.319963,-0.356792,-0.139893,-0.192215,-0.523949,-0.121952,-0.503665,...,-0.376761,-0.237079,-0.322901,-0.967510,-0.406922,-0.543061,0.766051,-0.151526,0.043085,-0.325006
75%,-0.342927,-0.190116,1.307094,-0.319963,-0.356792,-0.139893,-0.192215,-0.523949,-0.121952,-0.503665,...,-0.376761,-0.237079,-0.322901,1.033521,-0.406922,-0.543061,0.766051,0.653448,0.713871,0.646693
max,2.915907,5.259656,1.307094,3.125182,2.802590,7.147928,5.202207,1.908472,8.199477,1.985334,...,2.654049,4.217757,3.096745,1.033521,2.457333,1.841309,0.766051,3.263727,2.632318,3.182590


In [9]:
# 1. Prüfe auf konstante Features
constant_features = df_X.columns[df_X.std() < 0.01]
print(f"Nahezu konstante Features: {list(constant_features)}")

# 2. Prüfe Varianz der One-Hot Features
print(df_X.var().sort_values())

# 3. Prüfe ob alle One-Hot Gruppen eine Referenzkategorie haben
# (sollte eine Kategorie pro Gruppe fehlen)

NameError: name 'df_X' is not defined

In [7]:
# 2. Konstante zu Features hinzufügen
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

In [8]:
# 3. Modell auf Trainingsdaten fitten
print("Training Multinomial Logit Model...")
mnlogit_model = MNLogit(y_train, X_train_const)
result = mnlogit_model.fit(maxiter=100, method='bfgs')

print(result.summary())

Training Multinomial Logit Model...


c:\Users\david\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)


         Current function value: 1.492147
         Iterations: 100
         Function evaluations: 101
         Gradient evaluations: 101


c:\Users\david\anaconda3\Lib\site-packages\statsmodels\base\model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
c:\Users\david\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


ValueError: need covariance of parameters for computing (unnormalized) covariances

In [ ]:
# Matplot

# plt.scatter(x_train, y_train, c="orange")
# plt.scatter(x_test, y_test, c="blue")
# plt.plot(x_train, y_pred_train, c="red", ls=":")
# plt.plot(x_test, y_pred_test, c="red", ls="--")
# plt.title("Scatter Visualization and Plot for OLS-Model.")
# plt.savefig("UE_04_App2_ScatterVisualizationAndOlsModel.pdf")
# plt.close()
plt.scatter(df_train["x"], df_train["y"], c="orange", label="Train")
plt.scatter(df_test["x"], df_test["y"], c="blue", label="Test")

plt.plot(df_train["x"], y_pred_train, c="red", ls=":", label="Train Fit")
plt.plot(df_test["x"], y_pred_test, c="red", ls="--", label="Test Fit")

plt.title("Scatter Visualization and Plot for OLS-Model.")
plt.legend()
plt.savefig("UE_04_App2_ScatterVisualizationAndOlsModel.pdf")
plt.close()

plt.boxplot(
    [
        df_train["x"],
        df_train["y"],
        df_test["x"],
        df_test["y"]
    ],
    labels=["x_train", "y_train", "x_test", "y_test"]
)
plt.savefig("UE_04_App2_BoxPlot.pdf")
plt.close()